# DART 오픈 API
- 작성자: 고려대학교 경제학과 한치록 교수
- 본 매뉴얼은 로컬환경에서 활용하는 것을 목적으로 작성

[DART (전자공시) 오픈API][opendart]로부터 자료를 얻는 것에 대하여 논의한다. 본 매뉴얼은 `로컬환경(인터넷망 pc환경)`에서 활용하는 경우를 대상으로 작성한 것이다.

## 준비사항

### API Key

[API Key](https://opendart.fss.or.kr/mng/apiUsageStatusView.do)가 필요하며 [Open DART][opendart]에 계정을 만들고 API Key를 신청하여 얻을 수 있다. API Key는 40개 문자로 이루어진 문자열이다. 이하에서 `YOUR_API_KEY` 부분은 이 API Key를 의미한다.

### 공시대상회사 고유번호

API Key가 있으면 [공시대상회사의 고유번호](https://opendart.fss.or.kr/guide/detail.do?apiGrpCd=DS001&apiId=2019018) 정보를 얻을 수 있다. 이 정보는 다음 URL을 이용하여 다운로드한다다.
```
https://opendart.fss.or.kr/api/corpCode.xml?crtfc_key=YOUR_API_KEY
```

`YOUR_API_KEY` 부분은 실제 API key (40개 문자)로 치환해야 한다. 위 URL을 방문하면 파일을 내려받게 된다. 내려받은 파일의 이름은 "CORPCODE.zip"이며 압축을 풀면 "CORPCODE.xml" 파일이 생성된다. 이 XML 파일은 회사별로 회사코드(`corp_code`), 회사명(`corp_name`), 주식코드(`stock_code`), 수정일(`modify_date`) 정보를 포함한다. 이 XML 파일의 구조는 다음과 같다.

```xml
<?xml version="1.0" encoding="UTF-8"?>
<result>
    <list>
        <corp_code>00434003</corp_code>
        <corp_name>다코</corp_name>
        <stock_code> </stock_code>
        <modify_date>20170630</modify_date>
    </list>
    <list>
        <corp_code>00434456</corp_code>
        <corp_name>일산약품</corp_name>
        <stock_code> </stock_code>
        <modify_date>20170630</modify_date>
    </list>

```

[opendart]: https://opendart.fss.or.kr/

In [1]:
apikey = '----------------------'

def hidekey(url): return url.replace(f'{apikey}', 'YOUR_API_KEY') + ' (apikey hidden)'

import pandas as pd
import numpy as np

다운로드 받은 `CORPCODE.xml` 파일을 처리하여 [pandas DataFrame](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html)으로 변환해 두자. 다음 코드를 사용하면 된다.

In [2]:
# https://pandas.pydata.org/docs/reference/api/pandas.read_xml.html
df = pd.read_xml('CORPCODE.xml')
for v in df.columns:
    if isinstance(df[v][0],str): df[v] = df[v].str.strip()
df

,corp_code,corp_name,stock_code,modify_date
0,434003,다코,,20170630
1,434456,일산약품,,20170630
2,430964,굿앤엘에스,,20170630
3,432403,한라판지,,20170630
4,388953,크레디피아제이십오차유동화전문회사,,20170630
...,...,...,...,...
103803,1525205,정담유통,,20230228
103804,646510,미래에셋파트너스사호사모투자전문회사,,20230228
103805,1184822,미래에셋파트너스9호사모투자,,20230228
103806,755252,시니안,,20230228


위 코드의 문제는 `corp_code`가 숫자로 바뀐다는 것이다. 그래서 예를 들어 "00434003"이 숫자 434003으로 바뀐다. 하지만 `corp_code`는 "00434003"과 같이 8개 숫자로 이루어진 문자열이어야 하므로 이를 처리해 두어야 한다.

In [3]:
df['corp_code'] = ["%08d" % x for x in df['corp_code']]
df

,corp_code,corp_name,stock_code,modify_date
0,00434003,다코,,20170630
1,00434456,일산약품,,20170630
2,00430964,굿앤엘에스,,20170630
3,00432403,한라판지,,20170630
4,00388953,크레디피아제이십오차유동화전문회사,,20170630
...,...,...,...,...
103803,01525205,정담유통,,20230228
103804,00646510,미래에셋파트너스사호사모투자전문회사,,20230228
103805,01184822,미래에셋파트너스9호사모투자,,20230228
103806,00755252,시니안,,20230228


XML 파일을 읽는 데에는 시간이 좀 걸린다. 매번 이 작업을 반복할 수는 없으니 [pickle로 Serialize](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_pickle.html)하자.

In [4]:
df.to_pickle('CORPCODE.pkl')

`CORPCODE.pkl`로 저장되었다. XML 파일은 약 20 MB이고 pkl 파일은 4.4 MB이다.

Serialize된 pickle 파일을 읽으려면 다음과 같이 하면 된다.

```python
# https://pandas.pydata.org/docs/reference/api/pandas.read_pickle.html
import pandas as pd
df = pd.read_pickle('CORPCODE.pkl')
```

In [5]:
df = pd.read_pickle('CORPCODE.pkl')

다음과 같이 검색할 수 있다.

In [6]:
df.query("corp_name.str.contains('삼성전자')")

,corp_code,corp_name,stock_code,modify_date
66608,00258999,삼성전자서비스,,20220324
77769,01345812,삼성전자서비스씨에스,,20230125
88181,00252074,삼성전자판매,,20230220
90574,00126380,삼성전자,005930,20230110
99753,00366997,삼성전자로지텍,,20230811


위와 같이 회사 이름을 검색하여 고유번호(`corp_code`)를 구한다.

## 예시

### 보고서 접수번호

Open DART에서 실제 자료를 내려받을 경우 (대부분의 경우 불필요하지만) 드물게 고유번호 이외에도 접수번호(`rcept_no`)가 필요한 경우가 있다. 예를 들어 공시서류 원본파일과 재무제표 원본파일을 내려받을 때 접수번호가 필요하다. 접수번호(`rcept_no`)는 [공시정보](https://opendart.fss.or.kr/guide/main.do?apiGrpCd=DS001) 중 1번 [공시검색](https://opendart.fss.or.kr/guide/detail.do?apiGrpCd=DS001&apiId=2019001)을 이용하여 검색할 수 있다. 예를 들어 2021년 1월 1일부터 2023년 12월 31일까지 삼성전자(고유번호 00126380) 공시내역은 다음과 같이 얻을 수 있다.

In [7]:
import pandas as pd
import requests, json

def mkurl(service, **data):
    # apikey is globally defined
    s = f'https://opendart.fss.or.kr/api/{service}.json?crtfc_key={apikey}'
    for key,value in data.items():
        s = f'{s}&{key}={value}'
    return s

url = mkurl('list', corp_code='00126380', bgn_de='20210101', end_de='20231231')
# 다운로드
resp = requests.get(url)
# 정리
df = pd.DataFrame(resp.json()['list'])
df

,corp_code,corp_name,stock_code,corp_cls,report_nm,rcept_no,flr_nm,rcept_dt,rm
0,00126380,삼성전자,005930,Y,최대주주등소유주식변동신고서,20231211800418,삼성전자,20231211,유
1,00126380,삼성전자,005930,Y,[기재정정]임원ㆍ주요주주특정증권등소유상황보고서,20231211000100,최민기,20231211,
2,00126380,삼성전자,005930,Y,임원ㆍ주요주주특정증권등소유상황보고서,20231208000649,김수연,20231208,
3,00126380,삼성전자,005930,Y,임원ㆍ주요주주특정증권등소유상황보고서,20231206000226,최종근,20231206,
4,00126380,삼성전자,005930,Y,임원ㆍ주요주주특정증권등소유상황보고서,20231206000097,문태호,20231206,
5,00126380,삼성전자,005930,Y,임원ㆍ주요주주특정증권등소유상황보고서,20231206000087,선종우,20231206,
6,00126380,삼성전자,005930,Y,임원ㆍ주요주주특정증권등소유상황보고서,20231206000065,정진희,20231206,
7,00126380,삼성전자,005930,Y,임원ㆍ주요주주특정증권등소유상황보고서,20231205000302,김한조,20231205,
8,00126380,삼성전자,005930,Y,임원ㆍ주요주주특정증권등소유상황보고서,20231205000244,윤상용,20231205,
9,00126380,삼성전자,005930,Y,임원ㆍ주요주주특정증권등소유상황보고서,20231205000106,신병무,20231205,


위의 `rcept_no` 칼럼에 접수번호가 있다. 또한 위 다운받은 리스트는 처음 10개만 포함하며 전체 리스트가 아니다. 이후에

```python
resp.json()
```

라고 하면 `total_count`가 626개임을 알 수 있다. 총 626개 리스트를 모두 받기 위해서는 지금처럼 `total_count`를 구하고 나서, 이에 맞는 페이지 수를 계산하여 다운로드하면 된다. 자세한 내용의 설명은 불필요하며, 나중에 별도의 모듈에서 구현한다.

## Open DART API 서비스 목록

내려받을 수 있는 자료는 [Open DART 개발가이드 웹페이지](https://opendart.fss.or.kr/guide/main.do?apiGrpCd=DS001)에 나열되어 있다. 웹 페이지의 좌측에 6개 큰 분류가 있다. 이 6개 큰 분류는 다음과 같다.

* 공시정보 (4개 세부 서비스)
* 사업보고서 주요정보 (28개 세부 서비스)
* 상장기업 재무정보 (7개 세부 서비스)
* 지분공시 종합정보 (2개 세부 서비스)
* 주요사항보고서 주요정보 (36개 세부 서비스)
* 증권신고서 주요정보 (6개 세부 서비스)

각각의 큰 분류마다 2\~36개의 세부 서비스를 제공하여, 총 83가지 서비스를 제공한다. 이들 서비스 목록을 `DartAPI.xlsx` 파일에 정리하여 두었다(출처: Open DART 웹). 첫 번째 시트에 83가지 세부 서비스의 목록이 있고, 두 번째 시트에는 입력코드 내용의 일부가 정리되어 있다. 나중에 Open DART API가 변경되는 일이 발생하면 이 `DartAPI.xlsx` 파일을 업데이트하면 된다.

각 자료마다 요청인자 목록이 이 엑셀 파일에 있다. 예를 들어 회사채 미상환 잔액은 `cprndNrdmpBlce.json` 또는 `cprndNrdmpBlce.xml`로부터 얻는데, API 인증키(`crtfc_key`), 고유번호(`corp_code`), 사업연도(`bsns_year`), 보고서 코드(`reprt_code`)가 필요하다. 보고서 코드는 2번째 시트에 설명되어 있다. 예를 들어 사업보고서는 11011이다.

## 변수 내려받기 예시

변수들은 다음과 같이 내려받을 수 있다. 예를 들어 삼성전자 2022년 사업보고서상 미상환잔액은 다음과 같이 얻는다.

In [8]:
ccode = '00126380' # 삼성전자
url = mkurl('cprndNrdmpBlce', corp_code=ccode, bsns_year=2022, reprt_code=11011)
resp = requests.get(url)
pd.DataFrame(resp.json()['list'])

,rcept_no,corp_cls,corp_code,corp_name,sm,remndr_exprtn1,remndr_exprtn2,yy1_excess_yy2_below,yy2_excess_yy3_below,yy1_below,yy3_excess_yy4_below,yy4_excess_yy5_below,yy5_excess_yy10_below,yy10_excess
0,20230307000542,Y,00126380,삼성전자,"538,603,000,000",미상환잔액,공모,"6,337,000,000","513,257,000,000","6,337,000,000","6,337,000,000","6,335,000,000",-,-
1,20230307000542,Y,00126380,삼성전자,"65,000,000,000",미상환잔액,사모,-,"23,000,000,000",-,-,"42,000,000,000",-,-
2,20230307000542,Y,00126380,삼성전자,"603,603,000,000",미상환잔액,합계,"6,337,000,000","536,257,000,000","6,337,000,000","6,337,000,000","6,335,000,000",-,-


또 다른 예로, 2022년 삼성전자 사업보고서상 직원 현황은 다음과 같이 구한다.

In [9]:
url = mkurl('empSttus', corp_code=ccode, bsns_year=2022, reprt_code=11011)
resp = requests.get(url)
json = resp.json()
print([json['status'], json['message']])
pd.DataFrame(json['list'])

['000', '정상']


,rcept_no,corp_cls,corp_code,corp_name,sexdstn,fo_bbm,reform_bfe_emp_co_rgllbr,reform_bfe_emp_co_cnttk,reform_bfe_emp_co_etc,rgllbr_co,rgllbr_abacpt_labrr_co,cnttk_co,cnttk_abacpt_labrr_co,sm,avrg_cnwk_sdytrn,fyer_salary_totamt,jan_salary_am,rm
0,20230307000542,Y,00126380,삼성전자,남,DX,-,-,-,"38,023",-,333,-,"38,356",16.0,-,-,-
1,20230307000542,Y,00126380,삼성전자,여,DX,-,-,-,"11,948",223,94,-,"12,042",12.8,-,-,-
2,20230307000542,Y,00126380,삼성전자,남,DS,-,-,-,"51,079",-,126,-,"51,205",10.5,-,-,-
3,20230307000542,Y,00126380,삼성전자,여,DS,-,-,-,"19,777",181,24,-,"19,801",10.7,-,-,-
4,20230307000542,Y,00126380,삼성전자,남,성별합계,-,-,-,"89,102",-,459,-,"89,561",12.9,"12,408,336,000,000","143,000,000",-
5,20230307000542,Y,00126380,삼성전자,여,성별합계,-,-,-,"31,725",404,118,-,"31,843",11.5,"3,189,421,000,000","110,000,000",-


이상은 예시이다. 다음에는 구현된 모듈 `dartapi.py`에 대하여 설명한다.

# Open DART API 모듈

## 준비

주의할 점: 본 모듈은 CORPCODE.xml 파일로부터 정보를 읽어들여 CORPCODE.pkl 파일을 생성하도록 코딩되어 있다. 이 과정에서 만약 CORPCODE.pkl 파일이 없거나 더 오래되었으면 CORPCODE.xml 파일로부터 재차 읽어 자동으로 업데이트한다. 그런데 CORPCODE.zip 파일을 내려받아 압축을 풀면 CORPCODE.xml 파일의 최종 변경일시가 터무니없이 미래로 늦춰져 있을 수 있다. 그러면 본 모듈을 열 때마다 CORPCODE.xml 파일이 거듭 업데이트되어 속도가 현저히 저하된다. 이 문제가 발생하지 않도록 하려면 파일 변경일시를 현재 시점으로 맞추고 시작하도록 한다. 윈도우즈의 경우에는 파일을 복사하면 되는 것 같고, 맥 등 유닉스 계열에서는 `touch` 명령을 사용하면 된다.

```shell
touch CORPCODE.xml
```

이 작업을 꼭 먼저 실행하기 바란다.

## 모듈 시작하기

다음과 같이 모듈을 시작한다.

In [10]:
from bok_da.data import dartapi
dart = dartapi.DartAPI(apikey) # your API key here, optional: force = False|True

기업코드 정보는 `dart.corpcode` (데이터프레임)에, 서비스 리스트는 `dart.svcs` (데이터프레임)에 있다. API에 관한 모든 정보는 `dart.info` 데이터프레임을 참조하라. 요청인자들의 의미는 `dart.vocab`에 있다.

먼저 기업코드는 다음과 같이 검색할 수 있다.

In [11]:
dart.corpcode.query("corp_name.str.contains('삼성전자')")

,corp_code,corp_name,stock_code,modify_date
66608,00258999,삼성전자서비스,,20220324
77769,01345812,삼성전자서비스씨에스,,20230125
88181,00252074,삼성전자판매,,20230220
90574,00126380,삼성전자,005930,20230110
99753,00366997,삼성전자로지텍,,20230811


서비스 검색은 다음과 같이 한다.

In [12]:
dart.svcs.query("Service.str.contains('직원')")

,Category,Service,URL,Arguments
27,사업보고서 주요정보,직원 현황,empSttus,"crtfc_key, corp_code, bsns_year, reprt_code"


위 결과에서 `bsns_near`의 뜻이 궁금하면 다음과 같이 `dart.vocab`를 이용한다.

In [13]:
dart.vocab['bsns_year']

'사업연도'

상세한 정보는 `dart.info` 데이터프레임으로 저장되어 있으며, 다음과 같이 확인할 수 있다.

In [14]:
dart.info.query("Service.str.contains('직원')")

,Category,Service,URL,Arguments,Meaning,Required,Default,Source,Notes
109,사업보고서 주요정보,직원 현황,empSttus,crtfc_key,API 인증키,True,,,
110,사업보고서 주요정보,직원 현황,empSttus,corp_code,고유번호,True,,,
111,사업보고서 주요정보,직원 현황,empSttus,bsns_year,사업연도,True,,,
112,사업보고서 주요정보,직원 현황,empSttus,reprt_code,보고서 코드,True,,,


"직원"을 검색한 결과가 위에 있다. URL이 ".xml"로 끝나면 ".xml"만 사용 가능하며, 확장자가 없으면 ".json"과 ".xml"을 모두 사용할 수 있으나 본 모듈에서는 ".json"을 이용하도록 코딩하였다. 위 직원 현황 정보는 `empSttus`이므로 ".json"과 ".xml"을 모두 사용할 수 있지만 본 모듈에서는 ".json"을 사용하도록 코딩되어 있다. 위 결과로부터 '직원 현황' 데이터를 받기 위해서는 4개의 요청인자가 필요함을 알 수 있다.

실제 호출을 위한 URL은 다음 명령으로 만들 수 있다.

In [15]:
url = dart.mkurl(service = 'empSttus', corp_code = '00126380', bsns_year = 2022, reprt_code = 11011)
hidekey(url)

'https://opendart.fss.or.kr/api/empSttus.json?crtfc_key=YOUR_API_KEY&corp_code=00126380&bsns_year=2022&reprt_code=11011 (apikey hidden)'

위 URL로부터 데이터를 실제로 받는 방법은 앞에서 이미 설명하였으므로 반복하지 않겠다. 여러 연도에 걸쳐 데이터를 받을 때에는 각 연도마다 자료를 받으면 된다. 간편하게 받는 메쏘드를 구현해 놓았다(아래 '사업보고서 연도별 자료' 항목 참조).

## 데이터 다운로드 받기

### 1. 공시정보

#### 1.1 공시검색

공시검색은 주어진 기간 내 공시 내역을 검색하는 것이다. 기간은 시작일(`bgn_de`)과 종료일(`end_de`)로 주어지며, `doclist` 메쏘드로 구현하였는데, 캐시 저장은 이루어지지 않으며 메쏘드 호출 시 매번 DART로부터 다운로드가 이루어진다. 그러므로 인터넷 연결이 느리면 시간이 걸릴 수 있다.

In [16]:
dart.doclist('00126380', bgn_de='20210101', end_de='20210131')

,corp_code,corp_name,stock_code,corp_cls,report_nm,rcept_no,flr_nm,rcept_dt,rm
0,00126380,삼성전자,005930,Y,임원ㆍ주요주주특정증권등소유상황보고서,20210128000335,이창수,20210128,
1,00126380,삼성전자,005930,Y,매출액또는손익구조30%(대규모법인은15%)이상변경,20210128800077,삼성전자,20210128,유
2,00126380,삼성전자,005930,Y,수시공시의무관련사항(공정공시),20210128800075,삼성전자,20210128,유
3,00126380,삼성전자,005930,Y,현금ㆍ현물배당결정,20210128800069,삼성전자,20210128,유
4,00126380,삼성전자,005930,Y,연결재무제표기준영업(잠정)실적(공정공시),20210128800062,삼성전자,20210128,유
5,00126380,삼성전자,005930,Y,[기재정정]연결재무제표기준영업(잠정)실적(공정공시),20210128800050,삼성전자,20210128,유
6,00126380,삼성전자,005930,Y,횡령ㆍ배임사실확인,20210120800650,삼성전자,20210120,유
7,00126380,삼성전자,005930,Y,임원ㆍ주요주주특정증권등소유상황보고서,20210111000382,안규리,20210111,
8,00126380,삼성전자,005930,Y,최대주주등소유주식변동신고서,20210108800652,삼성전자,20210108,유
9,00126380,삼성전자,005930,Y,기업설명회(IR)개최(안내공시),20210108800573,삼성전자,20210108,유


#### 1.2 기업개황

기업개황 정보를 얻기 위해서는 기업코드(공시대상회사 고유번호, `corp_code`)가 필요하며, 기업코드는 `dart.corpcode`를 검색하여 찾을 수 있다. 주어진 기업코드에 대하여 기업개황을 다운로드받을 수 있는데, 매번 새로이 다운로드를 받으면 불필요한 인터넷 트래픽을 유발하므로, "cache_company" 폴더에 기업코드별로 파일이 저장되도록(캐시) 코딩하였다. 메쏘드 명은 `firminfo`이다. 이 메쏘드는 "cache_company" 폴더에서 기업코드를 검색하여 이 기업코드 json 파일이 존재하면 다운로드를 하지 않고 로컬 드라이브에서 파일을 읽는다. 만약 `force = True` 인자를 주면 강제로 재차 다운로드 받는다. "cache_company" 폴더를 지우면 캐시를 모두 없앤 것으로 생각하면 된다.

In [17]:
info = dart.firminfo('00126380') # download
info # dictionary

{'status': '000',
 'message': '정상',
 'corp_code': '00126380',
 'corp_name': '삼성전자(주)',
 'corp_name_eng': 'SAMSUNG ELECTRONICS CO,.LTD',
 'stock_name': '삼성전자',
 'stock_code': '005930',
 'ceo_nm': '한종희, 경계현',
 'corp_cls': 'Y',
 'jurir_no': '1301110006246',
 'bizr_no': '1248100998',
 'adres': '경기도 수원시 영통구  삼성로 129 (매탄동)',
 'hm_url': 'www.samsung.com/sec',
 'ir_url': '',
 'phn_no': '02-2255-0114',
 'fax_no': '031-200-7538',
 'induty_code': '264',
 'est_dt': '19690113',
 'acc_mt': '12'}

위 결과로부터 'stock_code'를 추출하려면 다음과 같이 한다.

In [18]:
info['stock_code']

'005930'

위에서는 하나 기업의 기업개황을 dict로 받았다. 여러 기업의 기업개황을 받는 경우에는 똑같이 `firminfo` 메쏘드를 사용하면서 기업코드 리스트를 입력인자로 주면 된다. 디폴트로 위아래로 붙은 DataFrame을 리턴하며(`byrow = True`), `byrow = False` 옵션을 주면 옆으로 붙은 DataFrame을 리턴한다.

In [19]:
dart.firminfo(['00126380', '00258999'], byrow = False) # compare with byrow = True (default)

,00126380,00258999
status,000,000
message,정상,정상
corp_code,00126380,00258999
corp_name,삼성전자(주),삼성전자 서비스주식회사
corp_name_eng,"SAMSUNG ELECTRONICS CO,.LTD","SAMSUNNG ELECTRONICS SERVICE CO., LTD"
stock_name,삼성전자,삼성전자서비스
stock_code,005930,
ceo_nm,"한종희, 경계현",송봉섭
corp_cls,Y,E
jurir_no,1301110006246,1301110049139


#### 1.3 공시보고서 원본파일

공시보고서 원본파일을 얻기 위해서는 접수번호가 필요하다. 접수번호는 공시검색을 통하여 얻을 수 있다. `getlist` 메쏘드로 구현하였는데, 이 메쏘드는 매번 DART로부터 정보를 다운로드 받으며 캐시 저장을 하지 않는다. 원할 경우 사용자가 캐시 저장을 하도록 다시 코딩을 하여야 할 것이다. 다음을 이용하여 삼성전자(corp_code는 00126380) 2021.1.1\~2021.1.31 기간에 접수된 보고서에 관한 정보를 얻는다. 캐시 저장 없이 매번 다운로드 받으므로 (인터넷 연결이 느리면) 시간이 소요될 것이다.

접수번호를 이용하여 공시보고서 원본파일을 받기 위한 URL을 생성할 수 있다. `url_for_origdoc` 메쏘드를 이용하면 된다. 아래 예에서는 API Key를 표시하지 않기 위해 `hidekey` 함수를 사용하였는데, 이 함수를 사용하지 않고 그냥 print하면 URL을 볼 수 있다. Jupyter 환경에서 작업할 경우 프린트된 URL을 클릭하면 파일을 다운로드받을 수 있다. 링크가 활성화되지 않는 환경에서는 생성된 URL을 웹브라우저 주소창에 복사해 넣고 엔터 키를 눌러 파일을 내려받는다. 한글을 UTF-8로 처리하지 않는 것으로 보이므로 주의가 필요하다.

In [20]:
print(hidekey(dart.url_for_origdoc("20210128000335")))

https://opendart.fss.or.kr/api/document.xml?crtfc_key=YOUR_API_KEY&rcept_no=20210128000335 (apikey hidden)


#### 1.4 고유번호

고유번호는 직접 다운로드 받아야 하는 것으로서 이 문서의 맨 앞 '준비사항'에 설명하였다.

### 2. 사업보고서 주요정보

'사업보고서 주요정보' 큰 분류에 포함된 다음 28개 변수는 각 연도별로 자료를 받을 수 있다.

In [21]:
dart.svcs.query("Category.str.contains('^사업보고서')")[['Service', 'URL']].reset_index(drop=True)

,Service,URL
0,조건부 자본증권 미상환 잔액,cndlCaplScritsNrdmpBlce
1,미등기임원 보수현황,unrstExctvMendngSttus
2,회사채 미상환 잔액,cprndNrdmpBlce
3,단기사채 미상환 잔액,srtpdPsndbtNrdmpBlce
4,기업어음증권 미상환 잔액,entrprsBilScritsNrdmpBlce
5,채무증권 발행실적,detScritsIsuAcmslt
6,사모자금의 사용내역,prvsrpCptalUseDtls
7,공모자금의 사용내역,pssrpCptalUseDtls
8,이사·감사 전체의 보수현황(주주총회 승인금액),drctrAdtAllMendngSttusGmtsckConfmAmount
9,이사·감사 전체의 보수현황(보수지급금액 - 유형별),drctrAdtAllMendngSttusMendngPymntamtTyCl


이 중 예를 들어 '직원현황'("empSttus") 자료를 일정 기간에 대하여 받는 방법은 다음과 같다. 우선 어떤 요청인자가 필요한지 한번 더 확인하자.

In [22]:
dart.info.query("Service.str.contains('^직원')")

,Category,Service,URL,Arguments,Meaning,Required,Default,Source,Notes
109,사업보고서 주요정보,직원 현황,empSttus,crtfc_key,API 인증키,True,,,
110,사업보고서 주요정보,직원 현황,empSttus,corp_code,고유번호,True,,,
111,사업보고서 주요정보,직원 현황,empSttus,bsns_year,사업연도,True,,,
112,사업보고서 주요정보,직원 현황,empSttus,reprt_code,보고서 코드,True,,,


위에 사용 입력인자들이 있다. 사실 '사업보고서 주요정보' 28개 세부 서비스는 모두 동일한 입력인자를 요구한다. 보고서 코드(`reprt_code`)가 무엇인지 한번 더 확인하려면 `dart.codes`를 검색하면 된다. 참고로, `dart.codes` 데이터프레임은 DartAPI.xlsx 파일에 저장된 정보를 바탕으로 만들어지며, DART 서버로부터 다운받지 않는다.

In [23]:
dart.codes.query("구분.str.contains('reprt_code')")

,구분한글,구분,코드,의미
19,보고서 코드,reprt_code,11013,1분기보고서
20,보고서 코드,reprt_code,11012,반기보고서
21,보고서 코드,reprt_code,11014,3분기보고서
22,보고서 코드,reprt_code,11011,사업보고서


이제 사업보고서(보고서 코드 "11011", 디폴트 인자)로부터 삼성전자(corp_code "00126380")의 2022년 직원 현황("empSttus")을 내려받자. 이 정보는 내역별로 연도별로 1개 존재하는 것이므로 캐시로 저장한다. 캐시 폴더는 "annual_cache"이다. 캐시 파일명은 이 예에서 "empSttus-00126380-2022-11011.json"이다. `force=True` 옵션을 주면 무조건 새로 다운로드 받아서 캐시로 저장한다.

In [24]:
dart.get_annual('empSttus', '00126380', [2021,2022]) # default는 reprt_code = 11011

,rcept_no,corp_cls,corp_code,corp_name,sexdstn,fo_bbm,reform_bfe_emp_co_rgllbr,reform_bfe_emp_co_cnttk,reform_bfe_emp_co_etc,rgllbr_co,rgllbr_abacpt_labrr_co,cnttk_co,cnttk_abacpt_labrr_co,sm,avrg_cnwk_sdytrn,fyer_salary_totamt,jan_salary_am,rm,year
0,20220308000798,Y,00126380,삼성전자,남,CE,-,-,-,"10,075",-,58,-,"10,133",16.1,-,-,-,2021
1,20220308000798,Y,00126380,삼성전자,여,성별합계,-,-,-,"29,120",407,108,-,"29,228",11.3,"3,161,517,000,000","115,000,000",-,2021
2,20220308000798,Y,00126380,삼성전자,여,CE,-,-,-,"2,077",72,9,-,"2,086",11.9,-,-,-,2021
3,20220308000798,Y,00126380,삼성전자,남,IM,-,-,-,"19,319",-,147,-,"19,466",14.4,-,-,-,2021
4,20220308000798,Y,00126380,삼성전자,여,IM,-,-,-,"6,941",72,19,-,"6,960",12.8,-,-,-,2021
5,20220308000798,Y,00126380,삼성전자,남,DS,-,-,-,"45,923",-,144,-,"46,067",10.0,-,-,-,2021
6,20220308000798,Y,00126380,삼성전자,여,DS,-,-,-,"17,813",182,22,-,"17,835",10.8,-,-,-,2021
7,20220308000798,Y,00126380,삼성전자,남,기타,-,-,-,"8,431",-,160,-,"8,591",15.0,-,-,-,2021
8,20220308000798,Y,00126380,삼성전자,여,기타,-,-,-,"2,289",81,58,-,"2,347",11.8,-,-,-,2021
9,20220308000798,Y,00126380,삼성전자,남,성별합계,-,-,-,"83,748",-,509,-,"84,257",12.2,"12,683,498,000,000","154,000,000",-,2021


### 3. 상장기업 재무정보

'상장기업 재무정보' 큰 분류에는 7개 서비스가 제공된다.

In [25]:
dart.svcs.query("Category.str.contains('상장기업 재무정보')").reset_index(drop = True)

,Category,Service,URL,Arguments
0,상장기업 재무정보,단일회사 주요계정,fnlttSinglAcnt,"crtfc_key, corp_code, bsns_year, reprt_code"
1,상장기업 재무정보,다중회사 주요계정,fnlttMultiAcnt,"crtfc_key, corp_code, bsns_year, reprt_code"
2,상장기업 재무정보,재무제표 원본파일(XBRL),fnlttXbrl.xml,"crtfc_key, rcept_no, reprt_code"
3,상장기업 재무정보,단일회사 전체 재무제표,fnlttSinglAcntAll,"crtfc_key, corp_code, bsns_year, reprt_code, f..."
4,상장기업 재무정보,XBRL택사노미재무제표양식,xbrlTaxonomy,"crtfc_key, sj_div"
5,상장기업 재무정보,단일회사 주요 재무지표,fnlttSinglIndx,"crtfc_key, corp_code, bsns_year, reprt_code, i..."
6,상장기업 재무정보,다중회사 주요 재무지표,fnlttCmpnyIndx,"crtfc_key, corp_code, bsns_year, reprt_code, i..."


#### 3.1 단일회사 주요계정, 3.2 다중회사 주요계정

맨 위의 2개(단일회사 주요계정, 다중회사 주요계정)는 '사업보고서 주요정보'와 동일한 형태로 자료를 받을 수 있다. 입력 인자도 동일하다.

In [26]:
dart.get_annual('fnlttSinglAcnt', '00126380', [2020,2021], '11011')

,rcept_no,reprt_code,bsns_year,corp_code,stock_code,fs_div,fs_nm,sj_div,sj_nm,account_nm,...,thstrm_amount,frmtrm_nm,frmtrm_dt,frmtrm_amount,bfefrmtrm_nm,bfefrmtrm_dt,bfefrmtrm_amount,ord,currency,year
0,20210309000744,11011,2020,00126380,005930,CFS,연결재무제표,BS,재무상태표,유동자산,...,"198,215,579,000,000",제 51 기,2019.12.31 현재,"181,385,260,000,000",제 50 기,2018.12.31 현재,"174,697,424,000,000",1,KRW,2020
1,20210309000744,11011,2020,00126380,005930,CFS,연결재무제표,BS,재무상태표,비유동자산,...,"180,020,139,000,000",제 51 기,2019.12.31 현재,"171,179,237,000,000",제 50 기,2018.12.31 현재,"164,659,820,000,000",3,KRW,2020
2,20210309000744,11011,2020,00126380,005930,CFS,연결재무제표,BS,재무상태표,자산총계,...,"378,235,718,000,000",제 51 기,2019.12.31 현재,"352,564,497,000,000",제 50 기,2018.12.31 현재,"339,357,244,000,000",5,KRW,2020
3,20210309000744,11011,2020,00126380,005930,CFS,연결재무제표,BS,재무상태표,유동부채,...,"75,604,351,000,000",제 51 기,2019.12.31 현재,"63,782,764,000,000",제 50 기,2018.12.31 현재,"69,081,510,000,000",7,KRW,2020
4,20210309000744,11011,2020,00126380,005930,CFS,연결재무제표,BS,재무상태표,비유동부채,...,"26,683,351,000,000",제 51 기,2019.12.31 현재,"25,901,312,000,000",제 50 기,2018.12.31 현재,"22,522,557,000,000",9,KRW,2020
5,20210309000744,11011,2020,00126380,005930,CFS,연결재무제표,BS,재무상태표,부채총계,...,"102,287,702,000,000",제 51 기,2019.12.31 현재,"89,684,076,000,000",제 50 기,2018.12.31 현재,"91,604,067,000,000",11,KRW,2020
6,20210309000744,11011,2020,00126380,005930,CFS,연결재무제표,BS,재무상태표,자본금,...,"897,514,000,000",제 51 기,2019.12.31 현재,"897,514,000,000",제 50 기,2018.12.31 현재,"897,514,000,000",13,KRW,2020
7,20210309000744,11011,2020,00126380,005930,CFS,연결재무제표,BS,재무상태표,이익잉여금,...,"271,068,211,000,000",제 51 기,2019.12.31 현재,"254,582,894,000,000",제 50 기,2018.12.31 현재,"242,698,956,000,000",17,KRW,2020
8,20210309000744,11011,2020,00126380,005930,CFS,연결재무제표,BS,재무상태표,자본총계,...,"275,948,016,000,000",제 51 기,2019.12.31 현재,"262,880,421,000,000",제 50 기,2018.12.31 현재,"247,753,177,000,000",21,KRW,2020
9,20210309000744,11011,2020,00126380,005930,CFS,연결재무제표,IS,손익계산서,매출액,...,"236,806,988,000,000",제 51 기,2019.01.01 ~ 2019.12.31,"230,400,881,000,000",제 50 기,2018.01.01 ~ 2018.12.31,"243,771,415,000,000",23,KRW,2020


#### 3.3 재무제표 원본파일(XBRL)

재무제표 원본파일(서비스 URL은 `fnlttXbrl.xml`)은 다음과 같은 방식으로 URL을 생성할 수 있다. 접수번호(`rcept_no`)와 보고서 코드(`reprt_code`)가 인자로 필요하다. 위 '공시보고서 원본파일' 다운로드 부분에 접수 문서 정보를 얻는 방법이 설명되어 있다(매번 DART 서버로부터 다운로드함). 예를 들어

```python
dart.doclist('00126380', bgn_de='20210101', end_de='20210131')
```

결과에 `rcept_no`가 있으므로 이를 참조하면 된다. `reprt_code`는 "11011" 등이다. 파일이 존재하지 않으면 존재하지 않는다고 할 것이다.

In [27]:
dart.info.query("Service.str.contains('^재무제표 원본파일')")

,Category,Service,URL,Arguments,Meaning,Required,Default,Source,Notes
137,상장기업 재무정보,재무제표 원본파일(XBRL),fnlttXbrl.xml,crtfc_key,API 인증키,True,,,
138,상장기업 재무정보,재무제표 원본파일(XBRL),fnlttXbrl.xml,rcept_no,접수번호,True,,list,
139,상장기업 재무정보,재무제표 원본파일(XBRL),fnlttXbrl.xml,reprt_code,보고서 코드,True,,,


In [28]:
# Slow - download from dart every time
dart.doclist('00126380', bgn_de='20210301', end_de='20210310')

,corp_code,corp_name,stock_code,corp_cls,report_nm,rcept_no,flr_nm,rcept_dt,rm
0,00126380,삼성전자,005930,Y,사업보고서 (2020.12),20210309000744,삼성전자,20210309,연
1,00126380,삼성전자,005930,Y,임원ㆍ주요주주특정증권등소유상황보고서,20210309000549,국민연금공단,20210309,
2,00126380,삼성전자,005930,Y,최대주주등소유주식변동신고서,20210308800940,삼성전자,20210308,유
3,00126380,삼성전자,005930,Y,임원ㆍ주요주주특정증권등소유상황보고서,20210305000674,국민연금공단,20210305,
4,00126380,삼성전자,005930,Y,임원ㆍ주요주주특정증권등소유상황보고서,20210303000376,박찬훈,20210303,
5,00126380,삼성전자,005930,Y,대규모기업집단현황공시[분기별공시(대표회사용)],20210302000645,삼성전자,20210302,공


In [29]:
# URL for 사업보고서 (2020.12)
print(hidekey(dart.mkurl('fnlttXbrl.xml', rcept_no = '20210309000744', reprt_code = '11011')))

https://opendart.fss.or.kr/api/fnlttXbrl.xml?crtfc_key=YOUR_API_KEY&rcept_no=20210309000744&reprt_code=11011 (apikey hidden)


위 기능은 빈번히 사용되지 않을 것으로 판단되어 별도의 메쏘드로 구현하지 않았다.

#### 3.4 단일회사 전체 재무제표

단일회사 전체 재무제표는 연도별로 받는다. 업체 고유번호, 사업연도, 보고서 코드 이외에 개별/연결구분 인자가 필요하다.

In [30]:
dart.info.query("Service.str.contains('^단일회사 전체 재무제표')")

,Category,Service,URL,Arguments,Meaning,Required,Default,Source,Notes
140,상장기업 재무정보,단일회사 전체 재무제표,fnlttSinglAcntAll,crtfc_key,API 인증키,True,,,
141,상장기업 재무정보,단일회사 전체 재무제표,fnlttSinglAcntAll,corp_code,고유번호,True,,,
142,상장기업 재무정보,단일회사 전체 재무제표,fnlttSinglAcntAll,bsns_year,사업연도,True,,,
143,상장기업 재무정보,단일회사 전체 재무제표,fnlttSinglAcntAll,reprt_code,보고서 코드,True,,,
144,상장기업 재무정보,단일회사 전체 재무제표,fnlttSinglAcntAll,fs_div,개별/연결구분,True,,,


개별/연결구분(`fs_div`)은 "OFS"(재무제표), "CFS"(연결재무제표) 중 하나이다. Default는 OFS이며, CFS는 `cfs = True` 인자를 이용하여 얻을 수 있다. 각 연도는 캐시로 저장되며, 강제로 재차 다운로드 받으려면 `force = True` 인자를 사용한다.

In [31]:
dart.get_annual_fs('00126380', [2021,2022])

,rcept_no,reprt_code,bsns_year,corp_code,sj_div,sj_nm,account_id,account_nm,account_detail,thstrm_nm,thstrm_amount,frmtrm_nm,frmtrm_amount,bfefrmtrm_nm,bfefrmtrm_amount,ord,currency,thstrm_add_amount,year
0,20220308000798,11011,2021,00126380,BS,재무상태표,ifrs-full_CurrentAssets,유동자산,-,제 53 기,73553416000000,제 52 기,73798549000000,제 51 기,72659080000000,1,KRW,NaN,2021
1,20220308000798,11011,2021,00126380,BS,재무상태표,ifrs-full_CashAndCashEquivalents,현금및현금성자산,-,제 53 기,3918872000000,제 52 기,989045000000,제 51 기,2081917000000,2,KRW,NaN,2021
2,20220308000798,11011,2021,00126380,BS,재무상태표,dart_ShortTermDepositsNotClassifiedAsCashEquiv...,단기금융상품,-,제 53 기,15000576000000,제 52 기,29101284000000,제 51 기,26501392000000,3,KRW,NaN,2021
3,20220308000798,11011,2021,00126380,BS,재무상태표,dart_ShortTermTradeReceivable,매출채권,-,제 53 기,33088247000000,제 52 기,24736740000000,제 51 기,26255438000000,4,KRW,NaN,2021
4,20220308000798,11011,2021,00126380,BS,재무상태표,dart_ShortTermOtherReceivables,미수금,-,제 53 기,1832488000000,제 52 기,1898583000000,제 51 기,2406795000000,5,KRW,NaN,2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
225,20230307000542,11011,2022,00126380,SCE,자본변동표,ifrs-full_Equity,기말자본,별도재무제표 [member],제 54 기,209416191000000,제 53 기,193193732000000,제 52 기,183316724000000,6,KRW,NaN,2022
226,20230307000542,11011,2022,00126380,SCE,자본변동표,ifrs-full_Equity,기말자본,자본 [member]|이익잉여금 [member],제 54 기,204388016000000,제 53 기,188774335000000,제 52 기,178284102000000,6,KRW,NaN,2022
227,20230307000542,11011,2022,00126380,SCE,자본변동표,ifrs-full_Equity,기말자본,자본 [member]|주식발행초과금,제 54 기,4403893000000,제 53 기,4403893000000,제 52 기,4403893000000,6,KRW,NaN,2022
228,20230307000542,11011,2022,00126380,SCE,자본변동표,ifrs-full_Equity,기말자본,자본 [member]|자본금 [member],제 54 기,897514000000,제 53 기,897514000000,제 52 기,897514000000,6,KRW,NaN,2022


#### 3.5 XBRL택사노미재무제표양식

재무제표 양식으로, 재무제표구분(`sj_div`)이 입력인자료 필요하다. [재무제표구분의 상세내역](https://opendart.fss.or.kr/guide/detail.do?apiGrpCd=DS003&apiId=2020001)을 참조하라. 파일은 `cache_others` 폴더에 "xbrlTaxonomy-BS1.json"과 같은 이름으로 ㅡ캐시 저장된다.

In [32]:
dart.xbrl_taxonomy('BS1')

,sj_div,bsns_de,account_id,account_nm,label_kor,label_eng,ifrs_ref,data_tp
0,BS1,20180701,ifrs_StatementOfFinancialPositionAbstract,StatementOfFinancialPositionAbstract,재무상태표 [abstract],Statement of financial position [abstract],,NaN
1,BS1,20180701,ifrs_AssetsAbstract,AssetsAbstract,자산 [abstract],Assets [abstract],,NaN
2,BS1,20180701,ifrs_CurrentAssets,CurrentAssets,유동자산,Current assets,K-IFRS 1001 문단 60,X
3,BS1,20180701,ifrs_CashAndCashEquivalents,CashAndCashEquivalents,현금및현금성자산,Cash and cash equivalents,"K-IFRS 1001 문단 54 (9),K-IFRS 1007 문단 45",X
4,BS1,20180701,ifrs_Cash,Cash,현금,Cash,"K-IFRS 1001 문단 54 (9),K-IFRS 1007 문단 45",X
...,...,...,...,...,...,...,...,...
514,BS1,20180701,dart_VoluntaryReserves,VoluntaryReserves,임의적립금,Voluntary Reserves,"K-IFRS 1001 문단 54 (18),K-IFRS 1001 문단 78 (5),K...",X
515,BS1,20180701,dart_RetainedEarningsBeforeAppropriationsAccum...,RetainedEarningsBeforeAppropriationsAccumulate...,미처분이익잉여금(미처리결손금),Retained earnings before appropriations(accumu...,"K-IFRS 1001 문단 54 (18),K-IFRS 1001 문단 78 (5),K...",X
516,BS1,20180701,ifrs_NoncontrollingInterests,NoncontrollingInterests,비지배지분,Non-controlling interests,K-IFRS 1001 문단 54 (17),X
517,BS1,20180701,ifrs_Equity,Equity,자본총계,Total equity,,NaN


#### 3.6 단일회사 주요 재무지표

[단일회사 주요 재무지표](https://opendart.fss.or.kr/guide/detail.do?apiGrpCd=DS003&apiId=2022001)는 연도별로 자료를 받으며, 고유번호(`corp_code`), 사업연도(`bsns_year`), 보고서 코드(`reprt_code`) 외에도 지표분류코드(`idx_cl_code`)가 필요하다. 지표분류코드는 다음 중 하나이다.

> 수익성지표 : M210000 안정성지표 : M220000 성장성지표 : M230000 활동성지표 : M240000

2023년 3분기 이후부터 정보가 제공된다. 사업체, 보고서 코드, 연도별, 지표분류별로 캐시 저장된다.

In [33]:
dart.get_annual_idx('00126380', 2023, '11014', idx_cl_code = 'M210000')

,reprt_code,bsns_year,corp_code,stock_code,idx_cl_code,idx_cl_nm,idx_code,idx_nm,idx_val,year
0,11013,2023,00126380,005930,M210000,수익성지표,M211100,세전계속사업이익률,4.783,2023
1,11013,2023,00126380,005930,M210000,수익성지표,M211200,순이익률,4.783,2023
2,11013,2023,00126380,005930,M210000,수익성지표,M211250,총포괄이익률,8.69,2023
3,11013,2023,00126380,005930,M210000,수익성지표,M211300,매출총이익률,29.757,2023
4,11013,2023,00126380,005930,M210000,수익성지표,M211400,매출원가율,70.243,2023
5,11013,2023,00126380,005930,M210000,수익성지표,M211550,ROE,2.544,2023
6,11013,2023,00126380,005930,M210000,수익성지표,M211800,판관비율,27.799,2023
7,11013,2023,00126380,005930,M210000,수익성지표,M212000,총자산영업이익률,0.829,2023
8,11013,2023,00126380,005930,M210000,수익성지표,M212100,총자산세전계속사업이익률,2.025,2023
9,11013,2023,00126380,005930,M210000,수익성지표,M212200,자기자본영업이익률,1.041,2023


#### 3.7 다중회사 주요 재무지표

다중회사 주요 재무지표를 얻기 위해서는 위의 단일회사 주요 재무지표에 `multi = True` 인자를 주면 된다. 2023년 3분기 이후부터 정보가 제공된다. 사업체, 보고서 코드, 연도별, 지표분류별로 캐시 저장된다.

In [34]:
dart.get_annual_idx('00126380', 2023, '11014', multi = True, idx_cl_code = 'M210000')

,reprt_code,bsns_year,corp_code,stock_code,idx_cl_code,idx_cl_nm,idx_code,idx_nm,idx_val,year
0,11013,2023,00126380,005930,M210000,수익성지표,M211100,세전계속사업이익률,4.783,2023
1,11013,2023,00126380,005930,M210000,수익성지표,M211200,순이익률,4.783,2023
2,11013,2023,00126380,005930,M210000,수익성지표,M211250,총포괄이익률,8.69,2023
3,11013,2023,00126380,005930,M210000,수익성지표,M211300,매출총이익률,29.757,2023
4,11013,2023,00126380,005930,M210000,수익성지표,M211400,매출원가율,70.243,2023
5,11013,2023,00126380,005930,M210000,수익성지표,M211550,ROE,2.544,2023
6,11013,2023,00126380,005930,M210000,수익성지표,M211800,판관비율,27.799,2023
7,11013,2023,00126380,005930,M210000,수익성지표,M212000,총자산영업이익률,0.829,2023
8,11013,2023,00126380,005930,M210000,수익성지표,M212100,총자산세전계속사업이익률,2.025,2023
9,11013,2023,00126380,005930,M210000,수익성지표,M212200,자기자본영업이익률,1.041,2023


### 4. 지분공시 종합정보

[지분공시 종합정보](https://opendart.fss.or.kr/guide/main.do?apiGrpCd=DS004)에는 다음 2개 서비스가 있다. 이들 서비스는 `corp_code`만을 입력 받으나, 지속적으로 변하는 것으로 보이며, 따라서 디폴트는 캐시 저장하지 않는다. `use_cache = True` 옵션을 주면 캐시 저장된 것을 사용한다.

In [35]:
dart.svcs.query("Category.str.contains('지분공시 종합정보')")

,Category,Service,URL,Arguments
39,지분공시 종합정보,대량보유 상황보고,majorstock,"crtfc_key, corp_code"
40,지분공시 종합정보,임원ㆍ주요주주 소유보고,elestock,"crtfc_key, corp_code"


메쏘드 명은 `get_stock_info`이며, 입력 인자로는 2개를 받는다. 첫 번째 인자는 서비스 종류("majorstock"은 대량보유 상황보고, "elestock"은 임원ㆍ주요주주 소유보고이다. 첫 번째 인자는 정수 0 또는 1로 줄 수도 있다. 이 경우 0은 대량보유 상황보고, 1은 임원ㆍ주요주주 소유보고이다.

In [36]:
dart.get_stock_info(0, corp_code = '00126380', use_cache = True)

,rcept_no,rcept_dt,corp_code,corp_name,report_tp,repror,stkqy,stkqy_irds,stkrt,stkrt_irds,ctr_stkqy,ctr_stkrt,report_resn
0,20220128000648,2022-01-28,00126380,삼성전자,일반,삼성물산,"1,261,674,789","-826,943",21.13,-0.02,"119,774,447",2.01,- 특별관계자 및 보유주식수 변동\n- 보유주식등에 관한 계약의 변경
1,20220323001259,2022-03-23,00126380,삼성전자,일반,삼성물산,"1,261,244,682","-430,107",21.13,-0.00,"119,774,447",2.01,- 특별관계자 및 보유주식수 변동
2,20220328001085,2022-03-28,00126380,삼성전자,일반,삼성물산,"1,241,311,463","-19,933,219",20.79,-0.34,"99,832,587",1.67,- 보유주식수 변동\n- 신탁 주식 처분
3,20220411003043,2022-04-11,00126380,삼성전자,약식,J.P.MORGANSECURITIESPLC,"14,858,917","14,858,917",0.22,0.22,-,-,신규보고의무 발생
4,20220422000748,2022-04-22,00126380,삼성전자,일반,삼성물산,"1,241,208,947","-102,516",20.79,-0.00,"99,832,587",1.67,- 보유주식수 변동\n- 보유주식등에 관한 계약의 변경
5,20220429001445,2022-04-29,00126380,삼성전자,일반,삼성물산,"1,241,202,321","-6,626",20.79,-0.00,"89,478,587",1.50,- 보유주식수 변동\n- 보유주식등에 관한 계약의 변경
6,20220506000547,2022-05-06,00126380,삼성전자,일반,삼성물산,"1,241,174,813","-27,508",20.79,-0.00,"98,413,587",1.65,- 보유주식수 변동\n- 보유주식등에 관한 계약의 변경
7,20220708000631,2022-07-08,00126380,삼성전자,일반,삼성물산,"1,241,131,025","-43,788",20.79,-0.00,"98,682,587",1.65,- 특별관계자 및 보유주식수 변동\n- 보유주식등에 관한 계약 관련 변경
8,20220722000492,2022-07-22,00126380,삼성전자,일반,삼성물산,"1,241,273,657","142,632",20.79,0.00,"98,282,587",1.65,- 보유주식수 변동\n- 보유주식등에 관한 계약의 변경
9,20220826000674,2022-08-26,00126380,삼성전자,일반,삼성물산,"1,241,213,913","-59,744",20.79,-0.00,"101,535,587",1.70,- 보유주식수 변동\n- 보유주식등에 관한 계약의 변경


### 5. 주요사항보고서 주요정보

[주요사항보고서 주요정보](https://opendart.fss.or.kr/guide/main.do?apiGrpCd=DS005)로 36개 서비스가 제공된다.

In [37]:
dart.svcs.query("Category.str.contains('^주요사항보고서')").reset_index(drop = True)

,Category,Service,URL,Arguments
0,주요사항보고서 주요정보,부도발생,dfOcr,"crtfc_key, corp_code, bgn_de, end_de"
1,주요사항보고서 주요정보,영업정지,bsnSp,"crtfc_key, corp_code, bgn_de, end_de"
2,주요사항보고서 주요정보,회생절차 개시신청,ctrcvsBgrq,"crtfc_key, corp_code, bgn_de, end_de"
3,주요사항보고서 주요정보,해산사유 발생,dsRsOcr,"crtfc_key, corp_code, bgn_de, end_de"
4,주요사항보고서 주요정보,유상증자 결정,piicDecsn,"crtfc_key, corp_code, bgn_de, end_de"
5,주요사항보고서 주요정보,무상증자 결정,fricDecsn,"crtfc_key, corp_code, bgn_de, end_de"
6,주요사항보고서 주요정보,유무상증자 결정,pifricDecsn,"crtfc_key, corp_code, bgn_de, end_de"
7,주요사항보고서 주요정보,감자 결정,crDecsn,"crtfc_key, corp_code, bgn_de, end_de"
8,주요사항보고서 주요정보,채권은행 등의 관리절차 개시,bnkMngtPcbg,"crtfc_key, corp_code, bgn_de, end_de"
9,주요사항보고서 주요정보,소송 등의 제기,lwstLg,"crtfc_key, corp_code, bgn_de, end_de"


위에서 볼 수 있는 것처럼 모두 기업코드(`corp_code`)외에 시작일(`bgn_de`)과 종료일(`end_de`)을 입력인자로 사용하게 되어 있다. 

In [38]:
dart.get_major_info('dfOcr', '00112819', '20190101', '20191231')

,rcept_no,corp_cls,corp_code,corp_name,df_cn,df_amt,df_bnk,dfd,df_rs
0,20191107000327,E,00112819,데코앤에프,당사발행 만기어음 부도,"440,558,786",신한은행 광교지점,2019년 11월 07일,"- 부도사유 : 예금부족\n- 부도경위 : 당사가 발행한 전자어음 440,558,7..."


### 6. 증권신고서 주요정보

증권신고서 주요정보로 다음 6가지 세부 서비스가 있다.

In [39]:
dart.svcs.query("Category.str.contains('^증권신고서')").reset_index(drop = True)

,Category,Service,URL,Arguments
0,증권신고서 주요정보,주식의포괄적교환·이전,extrRs,"crtfc_key, corp_code, bgn_de, end_de"
1,증권신고서 주요정보,합병,mgRs,"crtfc_key, corp_code, bgn_de, end_de"
2,증권신고서 주요정보,증권예탁증권,stkdpRs,"crtfc_key, corp_code, bgn_de, end_de"
3,증권신고서 주요정보,채무증권,bdRs,"crtfc_key, corp_code, bgn_de, end_de"
4,증권신고서 주요정보,지분증권,estkRs,"crtfc_key, corp_code, bgn_de, end_de"
5,증권신고서 주요정보,분할,dvRs,"crtfc_key, corp_code, bgn_de, end_de"


입력인자는 위 '주요사항보고서 주요정보'의 경우와 동일하다. 출력물 형태가 다르므로 별도 메쏘드로 구현하였다. `get_rs_info` 메쏘드이다.

In [40]:
rs = dart.get_rs_info('mgRs', '00109718', '20190101', '20191231')

In [41]:
rs.keys()

dict_keys(['일반사항', '발행증권', '당사회사에관한사항'])

In [42]:
rs['당사회사에관한사항']

,rcept_no,corp_cls,corp_code,corp_name,cmpnm,sen,tast,cpt,isstk_knd,isstk_cnt
0,20190402002153,Y,00109718,사조대림,주식회사 사조대림,존속회사,"720,876,000,000","29,822,000,000",보통주,"5,960,575"
1,20190402002153,Y,00109718,사조대림,주식회사 사조대림,존속회사,"720,876,000,000","29,822,000,000",우선주,"3,780"
2,20190402002153,Y,00109718,사조대림,주식회사 사조해표,소멸회사,"400,120,000,000","35,800,000,000",보통주,"7,160,000"
3,20190402002153,Y,00109718,사조대림,주식회사 사조해표,소멸회사,"400,120,000,000","35,800,000,000",우선주,-
